In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import geopandas as gpd
import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient
from sklearn.metrics import mean_absolute_error, r2_score

import sys

sys.path.append("../functions")

import knn

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

# Strategien

# Brukerinput

#### Eksempel:
###### 1
879263662


In [ ]:
aar = input("Årgang: ")

enhet = input("orgnr_foretak #: ")

In [ ]:
enhet_df, merged_gdf, reg_type_02_df, adjusted_result_df, naermeste_naboer = knn.knn(
    aar, enhet
)

# Kartlegging av næringen

In [ ]:
sg.explore(merged_gdf, "omsetning")

# Kartlegging av foretaket

In [ ]:
sg.explore(enhet_df, "omsetning")

# Å finne nærmeste naboer

In [ ]:
sg.explore(naermeste_naboer, "orgnr_foretak")

# Evaluering av resultater for reg_type 2 virksomheter

In [ ]:
# Calculate Mean Absolute Error (MAE) using the adjusted predictions
mae_new_predicted = mean_absolute_error(
    reg_type_02_df["actual_omsetning"], reg_type_02_df["new_predicted"]
)
r2_new_predicted = r2_score(
    reg_type_02_df["actual_omsetning"], reg_type_02_df["new_predicted"]
)
print(
    f"Mean Absolute Error (MAE) / Gjennomsnittlig absolutt feil for reg_type 2: {mae_new_predicted}"
)
print(f"R-kvadrert (R2) for reg_type 2: {r2_new_predicted}")

# # Evaluering av resultater for reg_type 2 virksomheter

In [ ]:
mae_new_predicted = mean_absolute_error(
    adjusted_result_df["actual_omsetning"], adjusted_result_df["new_predicted"]
)
r2_new_predicted = r2_score(
    adjusted_result_df["actual_omsetning"], adjusted_result_df["new_predicted"]
)
print(
    f"Mean Absolute Error (MAE)/ Gjennomsnittlig absolutt feil for hele næringen: {mae_new_predicted}"
)
print(f"R-squared (R2) for hele næringen: {r2_new_predicted}")

# Resultater 

In [ ]:
reg_type_02_df

In [ ]:
# need to make sure the unit is in the df unless it destroys the results too much

enhet_resultater = reg_type_02_df[reg_type_02_df["orgnr_foretak"] == enhet]
enhet_resultater

# Veien videre

#### Mean Absolute Error
Mean Absolute Error (MAE) for reg_type using new_predicted: 79725.23541605708
#### R-squared
R-squared (R2) for reg_type 2 using new_predicted: 0.8070814476089739

#### Mean Absolute Error
Mean Absolute Error (MAE) for the whole naring new_predicted: 17144.15600713256
#### R-squared
R-squared (R2) for the whole naring using new_predicted: 0.8580718457675715

In [ ]:
# Improving the model

In [ ]:
# Adding more features

In [ ]:
noku - ml / src / notebooks / Befolkning.xlsx

In [ ]:
testdatasti = "ssb-prod-dapla-felles-data-delt/GIS/testdata"
kommuner = sg.read_geopandas(f"{testdatasti}/enkle_kommuner.parquet")
pop = pd.read_excel("Befolkning.xlsx")
inntekt = pd.read_excel("inntekt.xlsx")
pop["KOMMUNENR"] = pop["KOMMUNENR"].astype("object")
inntekt["KOMMUNENR"] = inntekt["KOMMUNENR"].astype("object")

kommuner["KOMMUNENR"] = kommuner["KOMMUNENR"].str.replace('"', "").astype(str)
pop["KOMMUNENR"] = pop["KOMMUNENR"].astype(str)
pop["KOMMUNENR"] = pop["KOMMUNENR"].astype(str).str.zfill(4)

inntekt["KOMMUNENR"] = inntekt["KOMMUNENR"].astype(str)
inntekt["KOMMUNENR"] = inntekt["KOMMUNENR"].astype(str).str.zfill(4)


kommuner = kommuner.merge(pop, on="KOMMUNENR", how="left")
kommuner = kommuner.merge(inntekt, on="KOMMUNENR", how="left")

In [ ]:
m = sg.ThematicMap(enhet_df, kommuner, column="befolkning", size=15)
m.title = "Befolkning i kommunene"
m.plot()

In [ ]:
m = sg.ThematicMap(enhet_df, kommuner, column="inntekt", size=15)
m.title = "Inntekt i kommunene"
m.plot()